# What's in this notebook?


In [2]:
import pickle

with open('preprocessed_data.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    data = pickle.load(f)

data

,judgement,id,description,title,url,author_flair
id,,,,,,
1fy0bx,0,1fy0bx,I work in an office that requires me to wear a...,AItA: I like air conditioning and my coworkers...,http://www.reddit.com/r/AmItheAsshole/comments...,Asshole #1
1ytxov,1,1ytxov,[Here is the post in question](http://www.redd...,[AITA] I wrote an explanation in TIL and came ...,http://www.reddit.com/r/AmItheAsshole/comments...,None
1yu29c,1,1yu29c,"My parents are diabetic, morbidly obese, and a...",[AITA] Threw my parent's donuts away,http://www.reddit.com/r/AmItheAsshole/comments...,None
1yu8hi,0,1yu8hi,I was four.,I told a goth girl she looked like a clown.,http://www.reddit.com/r/AmItheAsshole/comments...,None
1yuc78,1,1yuc78,http://www.reddit.com/r/HIMYM/comments/1vvfkq/...,[AItA]: Argument I had with another redditor i...,http://www.reddit.com/r/AmItheAsshole/comments...,None
1yugsc,1,1yugsc,I love the musical *Les Miserables*. A coworke...,AITA Had a disagreement about Les Miserables w...,http://www.reddit.com/r/AmItheAsshole/comments...,None
1yuj95,1,1yuj95,Backstory: I'm a semi-professional competitive...,"[AITA] I 'hacked' our house router, changed th...",http://www.reddit.com/r/AmItheAsshole/comments...,None
1yukgb,0,1yukgb,2 Years ago my friend makes out with this girl...,[AITA] High School bullshit,http://www.reddit.com/r/AmItheAsshole/comments...,None
1yux10,1,1yux10,His name is Scoop. \n\nOver the two years I ha...,[ATIA] I permanently give a customer decaf.,http://www.reddit.com/r/AmItheAsshole/comments...,None
